<a href="https://colab.research.google.com/github/yahiamostafa/DeepLeaning-in-Cloud-Security/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from sklearn.utils import class_weight
import tensorflow.keras.layers as tkl

seed = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
fill data method to add the missed columns to test data 
to make the number of features in the train and the test equal.
'''
def fillData(trainInput , testInput):
    trainCols = trainInput.columns
    testCols = testInput.columns
    mask = np.in1d(trainCols , testCols)
    neededCols = np.where(~mask)[0]
    for col in neededCols[::-1]:
        testInput.insert(loc = int(col) , column = trainCols[col] , value = 0)
    return testInput

In [ ]:
'''
Normal Logistic Regression algorithm 
changing the number of iterations to improve the effciency.
'''
def logisticRegression(trainInput , trainOutput , testInput , testOutput):
    model = LogisticRegression(max_iter=300)
    model.fit(trainInput , trainOutput)
    print(model.score(trainInput , trainOutput))
    print(model.score(testInput , testOutput))

In [ ]:
'''
Data Prepocessing including
1- reading the csv file.
2- seperate the output from the file.
3- Applying OneHotEnocder to encode categorical features  into neumrical.
4- Normalizing data to make the gredient descent run faster. 
'''
def preprocessingBinaryClass(filePath , scaler = MinMaxScaler() , hotencoder = OneHotEncoder(), train = True , isCnn = False):
    dataframe = pd.read_csv(filePath , header = None)

     # to move the first Row to the last Position
    # firstRow = dataframe.iloc[0]
    # dataframe.drop(labels= 0 , axis= 0 , inplace= True)
    # dataframe = dataframe.append(firstRow , ignore_index = True)
    dataframe.columns = getFeaturesNames()
    dataframe.drop(['level'] , axis= 1 , inplace=True)
    output=dataframe['classification'].values
    output = output == 'normal'
    dataframe.drop(['classification']  , axis= 1,inplace=True)
    catFeatures =  dataframe.select_dtypes(include = "object")
    dataframe.drop(catFeatures.columns , axis= 1 , inplace= True)
    if train :
        scaler.fit(dataframe)
        hotencoder.fit(catFeatures)
    encodedDataframe = pd.DataFrame(hotencoder.transform(catFeatures).toarray())
    encodedDataframe.columns = hotencoder.get_feature_names_out()
    tempCols = dataframe.columns
    dataframe = pd.DataFrame(scaler.transform(dataframe) , columns = tempCols)
    dataframe = pd.concat([dataframe , encodedDataframe] , axis= 1)
    if isCnn:
      dataframe.drop(['num_outbound_cmds'] , axis= 1 , inplace=True)
      dataframe = dataframe.values.reshape(dataframe.shape[0] , 11 , 11 , 1)
    return dataframe , output , scaler ,hotencoder



In [ ]:
def outputMapping(arr):
    
    multiclass_attack={ 'normal': 'normal', 
        'probe': ['ipsweep.', 'nmap.', 'portsweep.','satan.', 'saint.', 'mscan.'], 
        'dos': ['back.', 'land.', 'neptune.', 'pod.', 'smurf.','teardrop.', 'apache2.', 'udpstorm.', 'processtable.','mailbomb.'], 
        'u2r': ['buffer_overflow.', 'loadmodule.', 'perl.', 'rootkit.','xterm.', 'ps.', 'sqlattack.'], 
        'r2l': ['ftp_write.', 'guess_passwd.', 'imap.', 'multihop.','phf.', 'spy.', 'warezclient.', 'warezmaster.','snmpgetattack.', 
                   'named.', 'xlock.', 'xsnoop.','sendmail.', 'httptunnel.', 'worm.', 'snmpguess.']}

    attacks = ['normal' , 'probe' , 'dos' , 'u2r' ,'r2l']
    
    for element in np.unique(arr):
        for attack in attacks:
            if element +"." in multiclass_attack[attack]:
                arr[ arr == element] = attack
    
    return arr

In [ ]:
'''
Data Prepocessing including
1- reading the csv file.
2- seperate the output from the file.
3- Applying OneHotEnocder to encode categorical features  into neumrical.
4- Normalizing data to make the gredient descent run faster. 
'''
def preprocessingFiveClasses(filePath , scaler = MinMaxScaler() , inEncoder = OneHotEncoder() , outEncoder = OneHotEncoder() , train = True , isCnn = False):
  dataframe = pd.read_csv(filePath , header = None)
  dataframe.columns = getFeaturesNames()
  dataframe.drop(['level'] , axis= 1 , inplace=True)
  output=outputMapping(pd.DataFrame(dataframe['classification']).values)
  dataframe.drop(['classification']  , axis= 1,inplace=True)
  catFeatures =  dataframe.select_dtypes(include = "object")
  dataframe.drop(catFeatures.columns , axis= 1 , inplace= True)
  out = output
  if train :
      scaler.fit(dataframe)
      inEncoder.fit(catFeatures)
      outEncoder.fit(output)
  output = pd.DataFrame(outEncoder.transform(output).toarray())
  encodedDataframe = pd.DataFrame(inEncoder.transform(catFeatures).toarray())
  encodedDataframe.columns = inEncoder.get_feature_names_out()
  tempCols = dataframe.columns
  dataframe = pd.DataFrame(scaler.transform(dataframe) , columns = tempCols)
  dataframe = pd.concat([dataframe , encodedDataframe] , axis= 1)
  if isCnn :
    dataframe.drop(['num_outbound_cmds'] , axis= 1 , inplace=True)
    dataframe = dataframe.values.reshape(dataframe.shape[0] , 11 , 11 , 1)
  return dataframe , output , out ,  scaler ,inEncoder , outEncoder


In [ ]:
def getFeaturesNames():
    features_name = pd.read_csv('drive/MyDrive/Field Names.csv')
    cols_name = features_name['duration'].tolist()
    cols_name.insert(0 , "duration")
    cols_name.append('classification')
    cols_name.append('level')
    return cols_name


In [ ]:
def MPLClassification(trainInput , trainOutput , testInput , testOutput):
    model = MLPClassifier(solver = 'adam' , alpha=0.0001 , max_iter= 200 , hidden_layer_sizes=(352 , 64 , 480 , 320 ,128) , random_state= 10 , batch_size=122)
    model.fit(trainInput , trainOutput)
    print(model.score(trainInput , trainOutput))
    print(model.score(testInput , testOutput))

In [ ]:
def predictTFresults(model , X , Y):
    pred = model.evaluate( X , Y , batch_size = 128)
    return {"Cost Function" : pred[0] , 'Accuracy' : pred[1]* 100}

In [ ]:
    def initTFparams(trainInput , trainOutput , lr , stddev , epochs ):

        print(f"lr {lr} and stddev {stddev}")
        np.random.seed(100)
        tf.random.set_seed(100)
        input_batch_size=122
        inputs = keras.Input(shape=(input_batch_size,), name="inputs")
        layer=layers.Dense(128, activation="relu", name="hidden_layer_1" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(inputs)
        layer=layers.Dense(256, activation="relu", name="hidden_layer_2" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        layer=layers.Dense(512, activation="relu", name="hidden_layer_3" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        layer=layers.Dense(1024, activation="relu", name="hidden_layer_4" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        layer=layers.Dense(256, activation="relu", name="hidden_layer_5" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        layer=layers.Dense(64, activation="relu", name="hidden_layer_6" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        outputs=layers.Dense(1,activation="sigmoid" ,name="output" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        model=keras.Model(inputs=inputs,outputs=outputs)
        model._name="MPLBinaryClassification"
        model.compile(loss='binary_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr),metrics=['accuracy'])
        model.fit(trainInput, trainOutput,epochs=epochs,batch_size=128 , validation_data=[testInput , testOutput])
        return model

In [ ]:
for i in range(0, 3):
  for j in range(7):
    lr = 10 ** (-1 * i)
    stddev = 10 ** (-1 * j)
    initTFparams2(trainInput , trainOutput , lr , stddev , 100 )


NameError: ignored

In [ ]:
    def initTFparams2(trainInput , trainOutput , lr , stddev , epochs ):

        print(f"lr {lr} and stddev {stddev}")
        np.random.seed(100)
        tf.random.set_seed(100)
        input_batch_size=122
        inputs = keras.Input(shape=(input_batch_size,), name="inputs")
        layer=layers.Dense(102, activation="relu", name="hidden_layer_1" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(inputs)
        layer=layers.Dense(50, activation="relu", name="hidden_layer_2" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        # layer=layers.Dense(512, activation="relu", name="hidden_layer_3" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        # # layer=layers.Dense(1024, activation="relu", name="hidden_layer_4" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        # # layer=layers.Dense(256, activation="relu", name="hidden_layer_5" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        # # layer=layers.Dense(64, activation="relu", name="hidden_layer_6" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        outputs=layers.Dense(1,activation="sigmoid" ,name="output" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
        model=keras.Model(inputs=inputs,outputs=outputs)
        model._name="MPLBinaryClassification"
        model.compile(loss='binary_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr),metrics=['accuracy'])
        model.fit(trainInput, trainOutput,epochs=epochs,batch_size=128 , validation_data=[testInput , testOutput])
        return model

In [ ]:
lr = 0.1
stddev = 1
epochs = 18
model = initTFparams2(trainInput , trainOutput , lr , stddev , epochs ) 
with open('drive/MyDrive/resultForMLPBinay.txt' , 'a') as file:
    file.write('___________________________________________________________________________________________________\n')
    file.write(f' lr = {lr} , stddev = {stddev} , epochs = {epochs}  , seed = {seed}  MLP2\n')
    file.write(str(predictTFresults(model , trainInput , trainOutput))+ '\n')
    file.write(str(predictTFresults(model , testInput , testOutput))+'\n')
    file.write(str(predictTFresults(model , test21Input , test21Output))+'\n')

lr 0.1 and stddev 1
Epoch 1/18
985/985 [==============================] - 4s 3ms/step - loss: 0.3016 - accuracy: 0.9809 - val_loss: 1.7804 - val_accuracy: 0.7738
Epoch 2/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0280 - accuracy: 0.9896 - val_loss: 2.0417 - val_accuracy: 0.7781
Epoch 3/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0237 - accuracy: 0.9907 - val_loss: 1.9974 - val_accuracy: 0.7834
Epoch 4/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0361 - accuracy: 0.9882 - val_loss: 2.3866 - val_accuracy: 0.7808
Epoch 5/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0323 - accuracy: 0.9896 - val_loss: 4.9364 - val_accuracy: 0.8075
Epoch 6/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0322 - accuracy: 0.9901 - val_loss: 4.2661 - val_accuracy: 0.7933
Epoch 7/18
985/985 [==============================] - 3s 3ms/step - loss: 0.0482 - accuracy: 0.9875 - val_loss: 4.9087 - val_acc

In [ ]:
def initTFparamsDiffClasses(trainInput , trainOutput ):   
    np.random.seed(seed)
    tf.random.set_seed(seed)
    input_batch_size=trainInput.shape[1]
    inputs = keras.Input(shape=(input_batch_size,), name="inputs")
    layer=layers.Dense(128, activation="relu", name="hidden_layer_1" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(inputs)
    layer=layers.Dense(256, activation="relu", name="hidden_layer_2" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    layer=layers.Dense(512, activation="relu", name="hidden_layer_3" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    layer=layers.Dense(1024, activation="relu", name="hidden_layer_4" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    layer=layers.Dense(256, activation="relu", name="hidden_layer_5" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    layer=layers.Dense(64, activation="relu", name="hidden_layer_6" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    outputs=layers.Dense(5,activation="softmax" ,name="output" , kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros())(layer)
    model=keras.Model(inputs=inputs,outputs=outputs)
    model._name="MPLDifferentClassification"
    model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
    model.fit(trainInput, trainOutput, epochs=100, batch_size=128 , validation_data=[testInput , testOutput])
    return model

In [ ]:
def initTFparamsDiffClasses2(trainInput , trainOutput , lr , stddev , epochs  ):   
    print(f"lr {lr} and stddev {stddev}")
    np.random.seed(100)
    tf.random.set_seed(100)
    input_batch_size=trainInput.shape[1]
    inputs = keras.Input(shape=(input_batch_size,), name="inputs")
    layer=layers.Dense(102, activation="relu", name="hidden_layer_1" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(inputs)
    layer=layers.Dense(50, activation="relu", name="hidden_layer_2" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
    outputs=layers.Dense(5,activation="softmax" ,name="output" , kernel_initializer=initializers.RandomNormal(stddev=stddev), bias_initializer=initializers.Zeros())(layer)
    model=keras.Model(inputs=inputs,outputs=outputs)
    model._name="MPLDifferentClassification"

    class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(out),
    y=out.reshape(-1)
    )
  
    class_weights = dict(zip([0,1,2,3,4],class_weights))

    model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate= lr),metrics=['accuracy'])
    model.fit(trainInput, trainOutput, epochs=epochs, batch_size=128 , validation_data=[testInput , testOutput] , class_weight = class_weights)
    return model

In [ ]:
lr = 0.1
stddev = 1
epochs = 44
seed = 100
model = initTFparamsDiffClasses2(trainInput , trainOutput , lr , stddev , epochs) 
with open('drive/MyDrive/resultForMLPMultiClassification.txt' , 'a') as file:
    file.write('___________________________________________________________________________________________________\n')
    file.write(f' lr = {lr} , stddev = {stddev} , epochs = {epochs}  , seed = {seed} MLP2 \n')
    file.write(str(predictTFresults(model , trainInput , trainOutput))+ '\n')
    file.write(str(predictTFresults(model , testInput , testOutput))+'\n')
    file.write(str(predictTFresults(model , test21Input , test21Output))+'\n')

lr 0.1 and stddev 1
Epoch 1/44
985/985 [==============================] - 4s 4ms/step - loss: 29.0623 - accuracy: 0.8708 - val_loss: 12.8987 - val_accuracy: 0.7445
Epoch 2/44
985/985 [==============================] - 3s 3ms/step - loss: 0.5160 - accuracy: 0.8863 - val_loss: 11.4964 - val_accuracy: 0.7909
Epoch 3/44
985/985 [==============================] - 3s 3ms/step - loss: 0.4552 - accuracy: 0.8992 - val_loss: 10.5889 - val_accuracy: 0.7751
Epoch 4/44
985/985 [==============================] - 3s 3ms/step - loss: 0.6257 - accuracy: 0.8868 - val_loss: 7.8169 - val_accuracy: 0.6717
Epoch 5/44
985/985 [==============================] - 3s 3ms/step - loss: 0.6071 - accuracy: 0.8683 - val_loss: 14.6944 - val_accuracy: 0.7549
Epoch 6/44
985/985 [==============================] - 3s 3ms/step - loss: 0.6174 - accuracy: 0.8697 - val_loss: 10.3504 - val_accuracy: 0.7328
Epoch 7/44
985/985 [==============================] - 3s 3ms/step - loss: 0.6725 - accuracy: 0.8325 - val_loss: 15.4134 - 

KeyboardInterrupt: ignored

In [ ]:
def conv2DBinary( X , Y , lr , stddev , epochs ,seed):

  print (lr , stddev , seed ,  sep =' ')
  np.random.seed(seed)
  tf.random.set_seed(seed)

  inputs = tkl.Input(shape=(11,11,1), name="inputs")

  layer = tkl.Conv2D(filters = 20 , input_shape = (5,5,20) , kernel_size=(2,2) , name ='conv2D1' )(inputs)
  layer = tkl.MaxPool2D(name = 'maxpool2D1')(layer)
  layer = tkl.BatchNormalization( name ='batchNorm1')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Conv2D(filters = 40 , input_shape = (2,2,40) , kernel_size=(2,2) , name ='conv2D2' )(layer)
  layer = tkl.MaxPool2D(name = 'maxpool2D2')(layer)
  layer = tkl.BatchNormalization( name ='batchNorm2')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Conv2D(filters = 60 , input_shape = (1,1,60) , kernel_size=(2,2) , name ='conv2D3' )(layer)
  layer = tkl.BatchNormalization( name ='batchNorm3')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Flatten()(layer)

  layer = tkl.Dense(64, name = 'FC1' ,kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm4')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Dense(80, name = 'FC2' , kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm5')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Dense(25, name = 'FC3' ,  kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm6')(layer)
  layer = tkl.Activation("relu")(layer)

  outputs = tkl.Dense(1 ,name="output" , activation = 'sigmoid' , kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  
  model=keras.Model(inputs=inputs,outputs=outputs)


  model.compile(loss='binary_crossentropy' , optimizer=tf.optimizers.Adam(learning_rate= lr) , metrics=['accuracy'] )
  model.fit( X , Y , epochs= epochs , batch_size=128 , validation_data = [test21Input,test21Output] )

  return model

SyntaxError: ignored

In [ ]:
model = conv2DMultiClass(trainInput , trainOutput , 0.0001 , 1 , 13)

print(predictTFresults(model , trainInput , trainOutput))
print(predictTFresults(model , testInput , testOutput))
print(predictTFresults(model , test21Input , test21Output))

0.0001 1
Epoch 1/13
985/985 [==============================] - 17s 15ms/step - loss: 1.6038 - accuracy: 0.7098 - val_loss: 2.6749 - val_accuracy: 0.4163
Epoch 2/13
985/985 [==============================] - 13s 13ms/step - loss: 0.5980 - accuracy: 0.8477 - val_loss: 2.1587 - val_accuracy: 0.4980
Epoch 3/13
985/985 [==============================] - 13s 13ms/step - loss: 0.4050 - accuracy: 0.8965 - val_loss: 2.0662 - val_accuracy: 0.5378
Epoch 4/13
985/985 [==============================] - 13s 13ms/step - loss: 0.3176 - accuracy: 0.9178 - val_loss: 1.9658 - val_accuracy: 0.5436
Epoch 5/13
985/985 [==============================] - 12s 13ms/step - loss: 0.2735 - accuracy: 0.9256 - val_loss: 1.7601 - val_accuracy: 0.5608
Epoch 6/13
985/985 [==============================] - 12s 13ms/step - loss: 0.2472 - accuracy: 0.9339 - val_loss: 1.5665 - val_accuracy: 0.5968
Epoch 7/13
985/985 [==============================] - 13s 13ms/step - loss: 0.1936 - accuracy: 0.9404 - val_loss: 1.5661 - val_

In [ ]:
def conv2DMultiClass( X , Y , lr , stddev , epochs) :

  print (lr , stddev ,  sep =' ')
  np.random.seed(100)
  tf.random.set_seed(100)

  inputs = tkl.Input(shape=(11,11,1), name="inputs")

  layer = tkl.Conv2D(filters = 20 , input_shape = (5,5,20) , kernel_size=(2,2) , name ='conv2D1' )(inputs)
  layer = tkl.MaxPool2D(name = 'maxpool2D1')(layer)
  layer = tkl.BatchNormalization( name ='batchNorm1')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Conv2D(filters = 40 , input_shape = (2,2,40) , kernel_size=(2,2) , name ='conv2D2' )(layer)
  layer = tkl.MaxPool2D(name = 'maxpool2D2')(layer)
  layer = tkl.BatchNormalization( name ='batchNorm2')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Conv2D(filters = 60 , input_shape = (1,1,60) , kernel_size=(2,2) , name ='conv2D3' )(layer)
  layer = tkl.BatchNormalization( name ='batchNorm3')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Flatten()(layer)

  layer = tkl.Dense(64, name = 'FC1' ,kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm4')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Dense(80, name = 'FC2' , kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm5')(layer)
  layer = tkl.Activation("relu")(layer)

  layer = tkl.Dense(25, name = 'FC3' ,  kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  layer = tkl.BatchNormalization( name ='batchNorm6')(layer)
  layer = tkl.Activation("relu")(layer)

  outputs = tkl.Dense(5 ,name="output" , activation = 'softmax' , kernel_initializer=initializers.RandomNormal(stddev= stddev))(layer)
  
  model=keras.Model(inputs=inputs,outputs=outputs)


  class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(out),
    y=out.reshape(-1)
  )
  
  class_weights = dict(zip([0,1,2,3,4],class_weights))

  model.compile(loss='categorical_crossentropy' , optimizer=tf.optimizers.Adam(learning_rate= lr) , metrics=['accuracy'] )
  model.fit( X , Y , epochs= epochs , batch_size=128 , validation_data = [test21Input,test21Output] , class_weight = class_weights )

  return model

In [ ]:
def CNN_BLSTM(input_train,output_train , lr , stddev):


    print(f"lr {lr} and stddev{ stddev}")
    np.random.seed(seed)
    tf.random.set_seed(seed)
    model = Sequential()
    init = initializers.RandomNormal(stddev= stddev)
    model.add(layers.Reshape((1,122), input_shape=(None,122)))
    model.add(keras.layers.Conv1D(filters=40,activation="tanh",name="Conv1d1",kernel_size=2,strides=1,padding="same",use_bias=True,bias_initializer=initializers.Zeros(),kernel_initializer=init))
    model.add(keras.layers.MaxPool1D(pool_size=2,strides=1,name="MaxPooling1",padding="same"))
    model.add(keras.layers.SpatialDropout1D(rate=0.1,name="Dropout_layer1"))
    model.add(keras.layers.Conv1D(filters=60,activation="tanh",name="Conv1d2",kernel_size=3,strides=1,padding="same",use_bias=True,bias_initializer=initializers.Zeros(),kernel_initializer=init))
    model.add(keras.layers.MaxPool1D(pool_size=3,strides=1,name="MaxPooling2",padding="same"))
    model.add(keras.layers.SpatialDropout1D(rate=0.1,name="Dropout_layer2"))
    model.add(keras.layers.Conv1D(filters=80,activation="tanh",name="Conv1d3",kernel_size=4,strides=1,padding="same",use_bias=True,bias_initializer=initializers.Zeros(),kernel_initializer=init))
    model.add(keras.layers.MaxPool1D(pool_size=4,strides=1,name="MaxPooling3",padding="same"))
    model.add(keras.layers.SpatialDropout1D(rate=0.1,name="Dropout_layer"))
    forward_layer = layers.LSTM(50, return_sequences=True,activation='tanh')
    backward_layer = layers.LSTM(50, activation='tanh', return_sequences=True,
                       go_backwards=True)
    model.add(keras.layers.Bidirectional(forward_layer, backward_layer=backward_layer))
    model.add(layers.Flatten())
    model.add(layers.Dense(1,activation="sigmoid",name="output", kernel_initializer=init, bias_initializer=initializers.Zeros()))
    model._name="CNNB_LSTM"
    # monitor=layers.EarlyStopping(monitor="val_loss",mode="min",patience=15,restore_best_weights=True)
    model.compile(loss='binary_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr),metrics=['accuracy'])
    model.fit(input_train,output_train,epochs=30,batch_size=128,validation_data=(test21Input,test21Output),callbacks=[])
    return model

In [ ]:
  
# trainInput , trainOutput ,  scaler , encoder = preprocessingBinaryClass('drive/MyDrive/KDDTrain+.csv')
# testInput , testOutput , *_ = preprocessingBinaryClass('drive/MyDrive/KDDTest+.csv' , scaler , encoder, False)
# test21Input , test21Output , *_ = preprocessingBinaryClass('drive/MyDrive/KDDTest-21.csv' ,  scaler , encoder,False)


trainInput , trainOutput , out , scaler , inEncoder , outEncoder = preprocessingFiveClasses('drive/MyDrive/KDDTrain+.csv' )
testInput , testOutput , *_ = preprocessingFiveClasses('drive/MyDrive/KDDTest+.csv' , scaler , inEncoder , outEncoder , False  )
test21Input , test21Output ,  *_ = preprocessingFiveClasses('drive/MyDrive/KDDTest-21.csv' , scaler , inEncoder , outEncoder , False )


